# generate a dataframe of sparse multi-hot clip labels for BirdSet evaluation sets
format will be a bit different than XCL training set

In [82]:
from opensoundscape import Audio, Spectrogram, CNN, BoxedAnnotations
import opensoundscape as opso
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path

from matplotlib import pyplot as plt
def figsize(w,h):
    plt.rcParams['figure.figsize']=[w,h]
figsize(15,5) #for big visuals
%config InlineBackend.figure_format = 'retina'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

import datasets
from tqdm.autonotebook import tqdm
from opensoundscape import annotations


Prepare and load the dataset

If this is your first time using it, it will download all of xeno-canto! (Consider using smaller datasets or XC subset)

For subsequent uses, just make sure to specify the same cache_dir so that it uses downloaded files


note that we are assuming the only annotated files are files that have >=1 annotation in the dataset

In [91]:
dataset_names = ['PER','NES','UHH','HSN','NBP','SSW','SNE','POW']#,'VOX']

In [92]:
cache_dir = "/home/kitzeslab/data/data_birdset/"

In [ ]:
for dataset_name in dataset_names:
    print(f"Processing dataset {dataset_name}")
    ds = datasets.load_dataset(
        "DBD-research-group/BirdSet",
        dataset_name,
        trust_remote_code=True,
        cache_dir=cache_dir,
    )
    classes = ds['test'].info.features['ebird_code'].names
    l=ds['test'].info.features['ebird_code']

    # aggregate bounding box annotations into a DataFrame
    records = []
    for i in range(len(ds["test"])):
        info= ds["test"][i]
        records.append({
            'audio_file':info['filepath'],
            'start_time': info['start_time'],
            'end_time': info['end_time'],
            'low_f': info['low_freq'],
            'high_f': info['high_freq'],
            'int_label': info['ebird_code'],
            'annotation': l.int2str(info['ebird_code']),
        })
    df = pd.DataFrame(records)
    df.to_csv(f"{cache_dir}/{dataset_name}_test_bbox.csv", index=False)

    
    # create labels on 3s and 5s non-overlapping clips
    ba = annotations.BoxedAnnotations(df,audio_files=df['audio_file'].unique())
    labels = ba.clip_labels(clip_duration=3,min_label_overlap=0.25,class_subset=classes,return_type='CategoricalLabels')
    labels_sparse_df = labels.multihot_df(sparse=True)
    labels_sparse_df.to_pickle(f"{cache_dir}/{dataset_name}_test_multihot_3s.pkl")

    ba = annotations.BoxedAnnotations(df,audio_files=df['audio_file'].unique())
    labels = ba.clip_labels(clip_duration=5,min_label_overlap=0.25,class_subset=classes,return_type='CategoricalLabels')
    labels_sparse_df = labels.multihot_df(sparse=True)
    labels_sparse_df.to_pickle(f"{cache_dir}/{dataset_name}_test_multihot_5s.pkl")

Processing dataset PER
Processing dataset NES
Processing dataset UHH


to use the labels in OpenSoundscape later: load the pickle
```python
labels = pd.read_pickle(f"{cache_dir}/POW_test_multihot_3s.pkl")
```